In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [3]:
### Load the trained model, scaler, and encoder pickles
model = load_model('model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    encoder = pickle.load(f)

with open('onehot_encoder_geography.pkl', 'rb') as f:
    encoder_geography = pickle.load(f)

In [4]:
## Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,  
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:
encoder_geo = encoder_geography.transform([[input_data['Geography']]])
geo_df = pd.DataFrame(encoder_geo, columns=encoder_geography.get_feature_names_out(['Geography']))
geo_df

d:\Python\ANN\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_df = pd.DataFrame([input_data]).drop(['Geography'], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Male,40,3,60000,2,1,1,50000


In [15]:
# Combine all features into a single DataFrame
input_data = pd.concat([input_df, geo_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
## Encode Gender
input_data['Gender'] = encoder.fit_transform(input_data['Gender'])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [19]:
## Scale the input data
input_data_scaled = scaler.transform(input_data)
input_data_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [20]:
## Predict the Churn
prediction = model.predict(input_data_scaled)
prediction

1/1 [==============================] - 0s 315ms/step


array([[0.05155579]], dtype=float32)

In [21]:
prediction_proba = prediction[0][0]
if prediction_proba > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_proba:.2f}")
else:
    print(f"The customer is unlikely to churn with a probability of {prediction_proba:.2f}")

The customer is unlikely to churn with a probability of 0.05
